# ***DATABASES WITH PYTHON CHEATSHEET***

---



the library 
`pandas` provides open source data analysis and manipulation tools.



In [ ]:
import pandas as pd

filename = "https://raw.githubusercontent.com/FredSansone/Data-Science/main/diabetes-train.csv"
# read_csv can be passed directly an URL
data_frame = pd.read_csv(filename) # stores a csv file in the object 'data_frame'
# note that read_csv can be passed directly an URL
data_frame # visualises the data frame on screen (or a portion thereof if the df is too big)
data_frame.shape # visualises data frame format as (<number of rows>,<number of cols>)

data_frame.columns # To see the name of all columns
data_frame.age[range(190,194)] # the columns are attributes of the data frame

sub_df = data_frame[(data_frame.age>50) & (data_frame.diabetes==1)] # select sub-data frame based on conditions...
#                                                                 ... each condition should be in ()

max(data_frame["age"]) # two ways to get statistics, and to call specific columns
data_frame.age.max()

data_frame.nlargest(10, ['bmi']) # the 10 rows with largest value in 'bmi' col
data_frame.nlargest(10, ['bmi','age']) # the 10 rows with largest value in 'bmi' col, with largest 'age' in case of a tie

data_frame.iloc[[0]] # {equiv: data_frame.iloc[[0],:]} sub-data frame of the row in position 0 (DataFrame type)
data_frame.iloc[0] # {equiv: data_frame.iloc[0,:]} gives same result, but in different format (Series type)
data_frame.iloc[0:10] # lines in range 0 to 9 (DataFrame type) (Python excludes upper extremum)
# data_frame.iloc[[0:10]] gives syntax error
data_frame.iloc[[0,2,4,1]] # lines 0,2,4,1 (DataFrame type)
data_frame.iloc[0,2] # entry 2 of Series line 0 (int64 type)
data_frame.iloc[[0,2,4,1,-1], [1,2,-2,-1]] # -k = row/col k from end (DataFrame type)
data_frame.iloc[:, 1:-1] # cols 1 to 2nd-to-last (DataFrame type) (Python excludes upper extremum)
data_frame.iloc[[*list(range(0, 4)), 6, 7]] # combine slices and lists as indices for iloc
# data_frame.iloc[[0:4, 6, 7] gives syntax error; must convert the slice 0:4 to a list; the * de-lists the inner list: 
# compare the following:
# [list(range(0,4)),6,7]# OUTPUT [[0, 1, 2, 3], 6, 7]
# [*list(range(0, 4)), 6, 7]# OUTPUT [0, 1, 2, 3, 6, 7]

data_frame.groupby("age")["bmi"].mean() # groups all the rows with same 'age' into one weird object,
# then computes the mean of the 'bmi' for each value of 'age'

agebins = pd.cut(data_frame["age"], bins=3, labels=("young", "mid age", "elderly")) # discretises the series data_frame["age"] 
# into 3 congruent bins, and gives bins ordered labels (Series type)
data_frame[["blood_pressure"]].groupby(agebins).agg(["mean", "median", "max"]) # df with indexes given by agebins and 3 cols made
# of bin statistics for 'blood_pressure' (DataFrame type)

data_frame[data_frame.bmi > 40].p_id.count() # self-explanatory; same as data_frame[data_frame.bmi > 40].<any col>.count()
# data_frame[data_frame.bmi > 40].count() (with no col) gives counts for each column

80

Nice code to get all the possible combinations of the columns of a data frame:

In [ ]:
import itertools

cols = data_frame.columns # or a selection of columns
for L in range(1, len(cols)+1):# for each cardinality between 1 and the number of columns of the data frame...
    for subset in itertools.combinations(cols, L): # ...for each subset of columns of that cardinality...
      cols_to_use = list(subset) # ...convert that subset to a list (so that it can be given as argument to the data frame)...
      sub_df = data_frame[cols_to_use] # ...get the sub-data frame corresponding to that list of columns
      # then do what you want to do with the selected columns, e.g.
      print(sub_df.iloc[[0]],"\n\n")
      


Less silly example of usage of the above: find the best choice of parameters in a regression analysis

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

record_of_vars = np.array([]) # initialization
record_of_Rsq = np.array([])
record_of_cv = np.array([])

dep_var = "diabetes" # or data_frame.columns[9]
y = data_frame[dep_var]
ind_var = data_frame.columns[1:9] # select the relevant columns to test as independent variables

for L in range(1, len(cols)+1):
    for subset in itertools.combinations(ind_var, L):
      vars_to_use = list(subset)
      X = data_frame[vars_to_use]

      record_of_vars = np.append(record_of_vars, ", ".join(vars_to_use))# records the choice of...
      # ...ind vars (", ".join(x) puts ", " in between each element of x and makes it a single string)

      model = sm.OLS(y, sm.add_constant(X)).fit() # ordinary least squares of y vs X
      Rsq = model.rsquared_adj # the adjusted R^2, i.e. with penalty for using more variables
      record_of_Rsq = np.append(record_of_Rsq, Rsq)

      cv_scores = cross_val_score(LinearRegression(), X, y, cv = 5) # cv is the number of partitions
      cv = np.mean(cv_scores)
      record_of_cv = np.append(record_of_cv, cv)

allData = pd.DataFrame({"columns": record_of_vars, "Rsq": record_of_Rsq, "cv": record_of_cv})
# creates a data frame with the syntax {<name of col 1>:<values of col 1>, ... , <name of col k>:<values of col k>}





Note that in this case the dependent variable is binary, so a linear regression is not appropriate; a logistic regression would be much better!

In [ ]:
interestingData = allData[(allData.Rsq >.28) & (allData.cv >.2)]
interestingData

,columns,Rsq,cv
104,"no_times_pregnant, glucose_concentration, bmi,...",0.281415,0.263081
169,"no_times_pregnant, glucose_concentration, bloo...",0.285907,0.266941
175,"no_times_pregnant, glucose_concentration, skin...",0.282487,0.261455
178,"no_times_pregnant, glucose_concentration, seru...",0.283631,0.262133
181,"no_times_pregnant, glucose_concentration, bmi,...",0.281534,0.255647
221,"no_times_pregnant, glucose_concentration, bloo...",0.286193,0.265266
224,"no_times_pregnant, glucose_concentration, bloo...",0.287957,0.266402
227,"no_times_pregnant, glucose_concentration, bloo...",0.287273,0.259156
228,"no_times_pregnant, glucose_concentration, skin...",0.283096,0.260586
231,"no_times_pregnant, glucose_concentration, skin...",0.282172,0.253538


Useful commands to work with series:

In [ ]:
occurrences = data_frame["age"].value_counts() # counts the occurrences of each value of the col "age" (Series type)
# the occurrences are sorted by descending value. 
sortedOccurrences = occurrences.sort_index() # this re-sorts by ascending index
#.sort_index(ascending = False) sorts by descending index
#.sort_values(ascending = True) sorts by ascending values
sortedOccurrences

21    48
22    56
23    36
24    37
25    39
26    26
27    26
28    26
29    25
30    17
31    19
32    12
33    15
34     7
35     7
36    13
37    17
38    11
39    10
40     9
41    16
42    13
43    10
44     7
45    15
46    11
47     6
48     4
49     4
50     7
51     5
52     7
53     4
54     5
55     3
56     3
57     4
58     4
59     3
60     3
61     2
62     3
63     4
64     1
65     2
66     3
67     3
68     1
69     2
70     1
72     1
81     1
Name: age, dtype: int64

In [ ]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'foo', 'bar'], 'rval': [3, 4, 5]})
pd.merge(left, right, on='key')

,key,lval,rval
0,foo,1,3
1,foo,1,4
2,bar,2,5
